In [ ]:
%matplotlib widget

In [ ]:
np.array([[0,159,226],
[213,93,72]])/255

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
#sns.set(palette="deep")
#cols = sns.color_palette()
cols = [[0.        , 0.62352941, 0.88627451],
        [0.83529412, 0.36470588, 0.28235294]]
from pathlib import Path

from matplotlib import pyplot as plt

from lotr import LotrExperiment, DATASET_LOCATION
from lotr.pca import pca_and_phase
from lotr.plotting import color_stack, despine, add_scalebar

def color_by_activation(roi_stack, exp, idx):
    activations = np.full(exp.n_rois, np.nan)
    activations[exp.hdn_indexes] = exp.traces[idx, exp.hdn_indexes]
    return color_stack(fixed_rois, activations, color_scheme="Greens")

In [ ]:
# List all experiments
master_path = Path(DATASET_LOCATION)

path = master_path / "210314_f1" / "210314_f1_natmov"# "210926_f0" / "210926_f0_gainmod" 

exp = LotrExperiment(path)
anatomy = exp.anatomy_stack
rois_stack = exp.rois_stack

pcaed, phase, pca, circle_params = pca_and_phase(exp.traces[:, exp.hdn_indexes])

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(np.arange(exp.n_pts), exp.traces[:, exp.hdn_indexes])
plt.plot(exp.behavior_log.t*exp.fn, exp.behavior_log.tail_sum + 7)
plt.plot(np.unwrap(phase))

In [ ]:
fixed_rois = exp.rois_stack.copy()
sizes = np.array([np.sum(fixed_rois==i) for i in exp.hdn_indexes])

for i, s in zip(exp.hdn_indexes, sizes):
    if s > 300:
        fixed_rois[fixed_rois==i] = -1
        fixed_rois[fixed_rois==i] = -1

In [ ]:
def plot_arrow(seg, ax=None, col="b", alpha=1, s=10, lw=1):
    ax.plot(seg[:, 0], seg[:, 1], lw=lw, c=col, alpha=alpha)
    ax.scatter(seg[0, 0], seg[0, 1], zorder=100, s=s, color=col, alpha=alpha, lw=0)
    
    ax.arrow(seg[-2, 0], seg[-2, 1], (seg[-1, 0] - seg[-2, 0]), (seg[-1, 1] - seg[-2, 1]),  
                 head_width=1, head_length=1.2, lw=lw, ec=col, fc=col, 
             zorder=100, alpha=alpha)

In [ ]:
bouts_idxs = dict(rt=[1165, 1010], lf=[3480, 280])

t_timepts = 4
f, pca_axs = plt.subplots(1, 4, figsize=(9, 2))
step = 20

pca_t_sl = slice(None, 4000)
for i in range(2):
    for j, side in enumerate(["rt", "lf"]):
        
add_scalebar(ax=pca_axs[0], xlen=7, ylen=7, xlabel="PC1", ylabel="PC2")

In [ ]:
gs = gridspec.GridSpec(4, 6, figure=fig)
axs_scat = [fig.add_subplot(gs[1:, :3]), fig.add_subplot(gs[1:, 3:])]
axs_comp = [fig.add_subplot(gs[0, 1:3]), fig.add_subplot(gs[0, 4:])]

In [ ]:
pca_axs.shape

In [ ]:
[i*2+j] 

In [ ]:
bouts_idxs = dict(rt=[1165, 1010], lf=[3480, 280])

t_timepts = 3
step = 25

f, axs_all = plt.subplots(2, 4, figsize=(8, 6))
pca_axs = axs_all[0, :]

y_crop = (250, 80)# slice()
for i in range(2):
# for i_col, start_idx in enumerate([1200, 3450, 1020, 270]):#8300, 270]):  #1020
    for j, side in enumerate(["rt", "lf"]):
        ax = pca_axs[i*2+j]       
        ax.plot(pcaed[pca_t_sl, 0], -pcaed[pca_t_sl, 1], lw=1, c=(0.9,)*3)
        ax.axis("equal")
            
        start_idx = bouts_idxs[side][i]
        seg = pcaed[start_idx: start_idx + step*t_timepts, :].copy()
        seg[:, 1] = -seg[:, 1]
        plot_arrow(seg, ax=ax, col=cols[j], s=8)
        despine(ax, "all")
        
        i_col = i*2+j
        start_idx = bouts_idxs[side][i]
        seg = (exp.behavior_log.t > (start_idx / exp.fn)) & \
                    (exp.behavior_log.t < (start_idx + step*t_timepts) / exp.fn)
        
        beh_ax = inset_axes(ax, width="100%", height="100%",
                   bbox_to_anchor=(0.3, -0.1, .8, .3),
                   bbox_transform=ax.transAxes, loc=2, borderpad=0)
        
        beh_ax.plot(exp.behavior_log[seg].t[::3], 
                    exp.behavior_log[seg].tail_sum[::3], lw=1, c=(0.4,)*3)
        
        add_scalebar(ax=beh_ax, xlen=3, ylen=2, ypos=-1.5,
                     xunits="s", yunits="rad", text_params=dict(fontsize=7))
        despine(beh_ax, "all")
        beh_ax.set(ylim=(-np.pi, np.pi))
        
        for d in range(2):
            despine(axs_all[d, i_col], "all")
        
        axs = axs_all[1, i_col]        
        
        colored = []
        for i_t in range(t_timepts):
            idx = start_idx + i_t*step
            colored.append(color_by_activation(fixed_rois, exp, idx)[:, :, y_crop[0]:y_crop[1]:-1, :])
        stacked = np.concatenate(colored, axis=2)
        axs.imshow(stacked.max(0).swapaxes(0, 1), aspect="equal")
        [axs.text(0, i, "a") for i in ]
        axs.set_ylabel(idx)
        despine(axs, "all")
            
add_scalebar(ax=axs_all[0, 0], xlen=7, ylen=7, xlabel="PC1", ylabel="PC2", text_params=dict(fontsize=8))

In [ ]:
plt.figure()
plt.imshow(colored[0])

In [ ]:
colored[0].shape

In [ ]:
color_by_activation(fixed_rois, exp, idx)[:, :, 250:80:-1, :].shape